<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/DAX_INDEX_PREDICTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
!pip install --upgrade tensorflow

import datetime
import yfinance as yf
import numpy as np
import pandas as pd
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, LSTM
from tensorflow import keras
from keras.layers import Dense, LSTM
from keras.models import Sequential, load_model
from sklearn.preprocessing import MinMaxScaler

# Download data for GER30 (DAX)
#symbol = "^GDAXI"
symbol = "^DJI"

start_date = "2010-01-01"
end_date = datetime.datetime.now().strftime("%Y-%m-%d")
df = yf.download(symbol, start=start_date, end=end_date, interval="1d")

# Preprocess data: Normalize 'Close' prices
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df['Close'].values.reshape(-1,1))

# Define a function to create sequences
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Prepare training and test datasets
seq_length = 60
X, y = create_sequences(df_scaled, seq_length)
train_size = int(len(df) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build LSTM Model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=32, epochs=10)

# Predict the trends for the next 5 days
predicted_prices = []
current_test_sequence = X_test[-1]  # Start with the last sequence in test data
for _ in range(5):
    current_test_sequence = current_test_sequence.reshape((1, seq_length, 1))
    predicted_price = model.predict(current_test_sequence)
    predicted_prices.append(predicted_price[0][0])
    # Update the sequence for the next prediction
    predicted_price = predicted_price.reshape((1, 1, 1))  # Reshape predicted_price
    current_test_sequence = np.append(current_test_sequence[:, 1:, :], predicted_price, axis=1)

# Inverse transform to get actual price predictions
predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1))

# Provide Buy, Sell, or Hold advice based on predictions
for i, price in enumerate(predicted_prices):
    action = "Hold"
    if i > 0 and price > predicted_prices[i-1]:
        action = "Buy"
    elif i > 0 and price < predicted_prices[i-1]:
        action = "Sell"
    print(f"Day {i+1}: Predicted Price: {price[0]}, Action: {action}")


[*********************100%%**********************]  1 of 1 completed
Epoch 1/10
88/88 [==============================] - 10s 54ms/step - loss: 0.0075
Epoch 2/10
88/88 [==============================] - 6s 68ms/step - loss: 3.3744e-04
Epoch 3/10
88/88 [==============================] - 7s 81ms/step - loss: 3.3686e-04
Epoch 4/10
88/88 [==============================] - 5s 55ms/step - loss: 3.2236e-04
Epoch 5/10
88/88 [==============================] - 6s 67ms/step - loss: 2.9877e-04
Epoch 6/10
88/88 [==============================] - 7s 84ms/step - loss: 2.9958e-04
Epoch 7/10
88/88 [==============================] - 5s 57ms/step - loss: 2.7067e-04
Epoch 8/10
88/88 [==============================] - 6s 65ms/step - loss: 2.6126e-04
Epoch 9/10
88/88 [==============================] - 8s 89ms/step - loss: 2.5248e-04
Epoch 10/10
1/1 [==============================] - 0s 40ms/step
Day 1: Predicted Price: 33671.25, Action: Hold
Day 2: Predicted Price: 33727.5, Action: Buy
Day 3: Predicted Price